# Import nhiều files excel cùng lúc hoặc nhiều sheets của 1 file excel vào Stata

Đào Huy Cừ - 09/06/2020 - Stata version 15.1

<h2><b> Import nhiều file excel trong cùng 1 folder </b></h2>

### Sử dụng câu lệnh xls2dta

In [1]:
// Cài đặt câu lệnh xls2dta
cap which xls2dta
if _rc ssc install xls2dta

In [2]:
mydir // lệnh gọi thư mục trong máy tính chứa datas
qui cd "$hrdata"
dir *.xls




  64.5k   6/17/20 11:00  201501.xls        
  65.0k   6/17/20 11:00  201502.xls        
  65.0k   6/17/20 11:00  201503.xls        
  66.0k   6/17/20 11:00  201504.xls        
  68.5k   6/17/20 11:00  201505.xls        


In [3]:
// Mô tả số liệu định import
local xlsfiles: dir"."files"*.xls"
foreach xls of local xlsfiles {
    import excel using `xls', describe
}




     Sheet | Range
  ---------+---------
    Sheet1 | A1:P229

     Sheet | Range
  ---------+---------
    Sheet1 | A1:P230

     Sheet | Range
  ---------+---------
    Sheet1 | A1:P230

     Sheet | Range
  ---------+---------
    Sheet1 | A1:P234

     Sheet | Range
  ---------+---------
    Sheet1 | A1:P243


In [4]:
// Import số liệu với xls2dta
local wd: pwd
// Tham số đưa vào khi import số liệu là folder chứa toàn bố số liệu cần import, chứ không phải từng tên file riêng
xls2dta, save("`wd'", replace): import excel "`wd'"

// Import số liệu vào stata và nối các file cùng định dạng với nhau
xls2dta, save("`wd'\completed.dta", replace): append



file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\201501.
> dta saved
file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\201502.
> dta saved
file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\201503.
> dta saved
file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\201504.
> dta saved
file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\201505.
> dta saved

(note: file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\
> completed.dta not found)
file D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data\complet
> ed.dta saved


In [5]:
use completed.dta, clear
count 



  1,166


### Sử dụng chương trình tự xây dựng (hiểu cơ bản về programming Stata)

In [6]:
set linesize 165
cap program drop myimport
program define myimport 
version 15
args ddang noifiles
global filenames : dir"."files"*.`ddang'"

foreach filename of global filenames {
	local filename1 = subinstr("`filename'"," ","",.)        
	local filename2 = subinstr("`filename1'",".`ddang'","",.) 
    if ("`ddang'" == "xlsx") | ("`ddang'"== "xls") {
        qui import excel using `"`filename'"', first all clear
        qui save `"`filename2'.dta"', replace
        clear
    }
    if ("`ddang'" == "csv") {
        qui import delimited using `"`filename'"', clear
        qui save `"`filename2'.dta"', replace
        clear
    }
}
// Nối các dta files lại với nhau
if "`noifiles'"!="" {
    local files:dir"."files"*.dta"
    clear
    local nfile = 1
    foreach f of local files {
        qui append using `f'
        cap gen varmark = `nfile'
        local ++nfile
        qui replace varmark = `nfile'
        }
    di as text "all dta files are appended, " _newline
}
end

In [7]:
// Ví dụ về 1 folder có chứa 48 files số liệu có cấu trúc giống nhau ở các biến số thu thập và format trong
dir *.csv

  18.1k   2/12/19 10:37  201501.csv        
  18.2k   2/12/19 10:37  201502.csv        
  18.2k   2/12/19 10:38  201503.csv        
  18.5k   2/12/19 10:38  201504.csv        
  19.1k   2/12/19 10:39  201505.csv        
  19.9k   2/12/19 10:39  201506.csv        
  20.4k   2/12/19 10:39  201507.csv        
  21.2k   2/12/19 10:40  201508.csv        
  21.7k   2/12/19 10:40  201509.csv        
  22.8k   2/12/19 10:40  201510.csv        
  23.6k   2/12/19 10:40  201511.csv        
  24.0k   2/12/19 10:41  201512.csv        
  24.9k   2/12/19 10:41  201601.csv        
  25.5k   2/12/19 10:41  201602.csv        
  26.9k   2/12/19 10:41  201603.csv        
  27.6k   2/12/19 10:42  201604.csv        
  28.1k   2/12/19 10:42  201605.csv        
  28.7k   2/12/19 10:42  201606.csv        
  29.3k   2/12/19 10:42  201607.csv        
  30.7k   2/12/19 10:43  201608.csv        
  32.4k   2/12/19 10:43  201609.csv        
  34.2k   2/12/19 10:43  201610.csv        
  36.8k   2/12/19 10:43  201611.

In [8]:
// Xóa những file dta trong folder
clear
local dtafiles: dir"."files"*.dta"
foreach v of local dtafiles {
    erase `v'
}

In [12]:
// Import tat ca cac files trong folder va luu lai file so lieu cung ten
myimport xls noifiles

variable empid is str6 in master but long in using data
    You could specify append's force option to ignore this string/numeric mismatch.  The using variable would then be treated as if it contained "".


r(106);


D:\NTP\Course-191204-CDC data visual\dashboard from youtube\HR Data


In [10]:
tab varmark


    varmark |      Freq.     Percent        Cum.
------------+-----------------------------------
         49 |     22,129      100.00      100.00
------------+-----------------------------------
      Total |     22,129      100.00
